In [28]:
# Install Required Libraries
!pip install --quiet --upgrade \
    bitsandbytes accelerate text-generation transformers langchainhub \
    sentencepiece gradio evaluate rouge-score nltk bert_score \
    fastapi uvicorn pyngrok flask flask-cors PyPDF2 torch langchain langchain_community
!pip install flask flask-ngrok

In [29]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.8 MB/s eta 0:00:00


In [2]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_community.document_loaders import PyPDFLoader
from io import BytesIO
import tempfile
import torch

In [21]:
!ngrok config add-authtoken 2pM3Fa6aFN50cWQ8mx90ikVf1rp_4L5QEHQkFcDJpr7C6fTs

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [30]:
# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

In [5]:
# Initialize the model using HuggingFace

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    trust_remote_code=True,
    device_map="auto"
)


text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype="bfloat16",
    trust_remote_code=True
)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [31]:
import os
import traceback
from flask import Flask, request, jsonify
from io import BytesIO
import tempfile
from langchain_community.document_loaders import PyPDFLoader

#def rearrange_text(text):
    # Custom logic to rearrange the text
    # Example: Sort words alphabetically
   # words = text.split()
    #rearranged_text = " ".join(sorted(words))
    #return rearranged_text

# Route to handle file upload
@app.route('/upload', methods=['POST'])
def upload_pdf():
    if 'file' not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files['file']
    if not file.filename.endswith('.pdf'):
        return jsonify({"error": "Only PDF files are allowed"}), 400

    try:
        # Load the PDF and extract text
        pdf_file = BytesIO(file.read())
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_pdf_file:
            temp_pdf_file.write(pdf_file.getvalue())
            temp_pdf_file_path = temp_pdf_file.name

        try:
            loader = PyPDFLoader(temp_pdf_file_path)
            documents = loader.load()
            extracted_text = " ".join([doc.page_content for doc in documents])

            # Apply rearrangement logic
            #rearranged_text = rearrange_text(extracted_text)

            return jsonify({
                "message": "PDF processed successfully!",
                "extracted_text": extracted_text
            }), 200

        finally:
            # Ensure the temporary file is deleted
            if os.path.exists(temp_pdf_file_path):
                os.unlink(temp_pdf_file_path)

    except Exception as e:
        # Log the error with a full traceback
        print(f"Error processing PDF: {e}")
        traceback.print_exc()
        return jsonify({"error": f"Failed to process PDF. Error: {str(e)}"}), 500


In [32]:
# Route to summarize text
@app.route('/summarize', methods=['POST'])
def summarize():
    data = request.json
    if not data or 'text' not in data:
        return jsonify({"error": "No text provided"}), 400

    input_text = data['text']

    # Template for summarization
    template = """
       You are a summarizer tasked with condensing the provided text into a high-quality summary presented as bullet points.
       Your response should start immediately with the summary, avoiding any introductory remarks.
       The summary must:
       Cover all essential contents of the lesson.
       Be comprehensive enough to replace studying the slides.
       Simplify long definitions while maintaining accuracy and understanding.
       Include any enumerations, examples, or key points in a structured and logical order.
       Content: {text}

       Your Answer:
       (Summary starts directly here in bullet-point format.)
       """
    prompt = template.replace("{text}", input_text)

    # Generate summary
    sequences = text_generator(
        prompt,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.3,
        num_return_sequences=1,
        repetition_penalty=1.2,
        return_full_text=False
    )

    generated_summary = sequences[0]['generated_text']
    return jsonify({"summary": generated_summary})


In [33]:
# Start Flask server
if __name__ == '__main__':
    from google.colab.output import eval_js
    from pyngrok import ngrok
    ngrok.kill()
    public_url = ngrok.connect("5000")

    print(f"Custom Ngrok URL: {public_url}")
    # Setup ngrok
    #ngrok_tunnel = ngrok.connect(5000)
    #print(f"Public URL: {ngrok_tunnel.public_url}")

Custom Ngrok URL: NgrokTunnel: "https://474f31a408b7.ngrok.app" -> "http://localhost:5000"


In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://474f31a408b7.ngrok.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [25/Nov/2024 19:34:59] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2024 19:34:59] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2024 19:35:47] "POST /upload HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
INFO:werkzeug:127.0.0.1 - - [25/Nov/2024 19:36:40] "POST /summarize HTTP/1.1" 200 -
